In [1]:
import pandas as pd

finetuned_dirname = "10-epoch-roberta-finetuned-phemernr2-rnr-check-finetune"

data = pd.read_csv("../../data/phemernr2_dataset_with_tvt.csv", sep=",")
# data = data[['tweet_text', 'tvt2', 'label2']]
# data.columns = ['tweet_text', 'tvt2', 'label']
print(data.shape)
data.head()

(6425, 5)


,tweet_id,tweet_text,label,tvt2,tvt2_1
0,552833795142209536,the east london mosque would like to offer its...,non-rumours,training,training
1,580318210609696769,breaking - a germanwings airbus a320 plane rep...,true,validation,testting
2,552798891994009601,reports that two of the dead in the #charliehe...,true,training,training
3,576790814942236672,after #putin disappeared russian tv no longer ...,non-rumours,validation,training
4,499678822598340608,saw #ferguson for myself. #justiceformichaelbr...,non-rumours,testting,testting


In [2]:
data['label'] = data['label'].replace(['true', 'unverfied', 'false'], 'rumors')

In [3]:
combined_data = data

In [4]:
import torch

class CustomTextDataset(torch.utils.data.dataset.Dataset):

    def __init__(self, texts, labels):
        self.labels = labels
        self.texts = texts
        self.attention_mask = None
        self.input_ids = None
        self.token_type_ids = None

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        sample = {
            "text": self.texts[idx],
            "label": self.labels[idx],
            "attention_mask": self.attention_mask[idx] if self.attention_mask else None,
            "input_ids": self.input_ids[idx] if self.input_ids else None,
        }
        return sample
    
    def tokenize(self, tokenizer):
        self.attention_mask = []
        self.input_ids = []
        self.token_type_ids = []

        for text in self.texts:
            token = tokenizer(text, padding="max_length", truncation=True)
            
            self.attention_mask.append(token['attention_mask'])
            self.input_ids.append(token['input_ids'])

c:\Users\msi_b\anaconda3\envs\pytorch-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
labels = []

labels_str = combined_data['label'].unique().tolist()
for i, d in combined_data.iterrows():
    if d['label'] == "non-rumours":
        lab = 1
    else:
        lab = 0
    labels.append(lab)
    
print(len(labels))
labels[:10]

6425


[1, 0, 0, 1, 1, 1, 1, 0, 1, 1]

In [6]:
train_dataset = CustomTextDataset(
    [d['tweet_text'] for i, d in combined_data.iterrows() if d['tvt2'] == 'training'],
    [labels[i] for i, d in combined_data.iterrows() if d['tvt2'] == 'training'])
test_dataset = CustomTextDataset(
    [d['tweet_text'] for i, d in combined_data.iterrows() if d['tvt2'] == 'validation'],
    [labels[i] for i, d in combined_data.iterrows() if d['tvt2'] == 'validation'])
train_dataset[0]

{'text': 'the east london mosque would like to offer its sincere condolences to the families of those killed during the #charliehebdo attacks (1/2)',
 'label': 1,
 'attention_mask': None,
 'input_ids': None}

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base")

In [8]:
# inputs = tokenizer(["you're stuck in a timewrap from 2004 though", "summa lumma dumma lumma"], padding="max_length", truncation=True)
# for k,v in inputs.items():
#     print(k)

In [9]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

train_dataset.tokenize(tokenizer)
test_dataset.tokenize(tokenizer)

In [10]:
print(len(train_dataset))
print(len(test_dataset))

4336
1462


### Fine Tuning

In [11]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("roberta-base",
                                                           output_hidden_states=False,
                                                           num_labels=2)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
from transformers import TrainingArguments

epochs = 10
batch_size = 8
save_steps = (round((len(train_dataset)/batch_size) + 0.49)) * epochs
# save_steps = 1_000_000

training_args = TrainingArguments(
    output_dir=f"../../data/models/{finetuned_dirname}",
    num_train_epochs=epochs,
    save_steps=save_steps,
    logging_steps=300,
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    evaluation_strategy="epoch",
    logging_strategy="epoch"
)

print(f"Save Steps : {save_steps}")

Save Steps : 5420


In [13]:
import numpy as np
from datasets import load_metric
# from copy import deepcopy

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# def compute_metrics(eval_pred):
#     metrics = ["accuracy", "recall", "precision", "f1"] #List of metrics to return
#     metric={}
#     for met in metrics:
#        metric[met] = load_metric(met)
#     logits, labels = eval_pred
#     predictions = np.argmax(logits, axis=-1)
#     metric_res={}
#     for met in metrics:
#        metric_res[met]=metric[met].compute(predictions=predictions, references=labels)[met]
#     return metric_res

C:\Users\msi_b\AppData\Local\Temp\ipykernel_9372\3274651076.py:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [14]:
# class CustomCallback(TrainerCallback):
    
#     def __init__(self, trainer) -> None:
#         super().__init__()
#         self._trainer = trainer
    
#     def on_epoch_end(self, args, state, control, **kwargs):
#         if control.should_evaluate:
#             control_copy = deepcopy(control)
#             self._trainer.evaluate(eval_dataset=self._trainer.train_dataset, metric_key_prefix="train")
#             return control_copy

In [15]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

In [16]:
import time

start = time.time()

train = trainer.train()

print(f"Execution Time : {round(time.time() - start)} seconds")

 10%|█         | 542/5420 [02:49<25:01,  3.25it/s]

{'loss': 0.4855, 'learning_rate': 9e-06, 'epoch': 1.0}


                                                  
 10%|█         | 542/5420 [03:06<25:01,  3.25it/s]

{'eval_loss': 0.4910690486431122, 'eval_accuracy': 0.8153214774281806, 'eval_runtime': 16.7671, 'eval_samples_per_second': 87.195, 'eval_steps_per_second': 10.914, 'epoch': 1.0}


 20%|██        | 1084/5420 [05:54<22:07,  3.27it/s] 

...

{'eval_loss': 0.8806429505348206, 'eval_accuracy': 0.8803009575923393, 'eval_runtime': 16.7676, 'eval_samples_per_second': 87.192, 'eval_steps_per_second': 10.914, 'epoch': 9.0}


100%|██████████| 5420/5420 [30:20<00:00,  3.23it/s]

{'loss': 0.0374, 'learning_rate': 0.0, 'epoch': 10.0}


                                                   
100%|██████████| 5420/5420 [30:37<00:00,  2.95it/s]

{'eval_loss': 0.9133529663085938, 'eval_accuracy': 0.8755129958960328, 'eval_runtime': 16.7655, 'eval_samples_per_second': 87.203, 'eval_steps_per_second': 10.915, 'epoch': 10.0}
{'train_runtime': 1837.6678, 'train_samples_per_second': 23.595, 'train_steps_per_second': 2.949, 'train_loss': 0.20009695982141248, 'epoch': 10.0}
Execution Time : 1838 seconds


In [17]:
trainer.evaluate()

100%|██████████| 183/183 [00:16<00:00, 10.97it/s]


{'eval_loss': 0.9133529663085938,
 'eval_accuracy': 0.8755129958960328,
 'eval_runtime': 16.7832,
 'eval_samples_per_second': 87.111,
 'eval_steps_per_second': 10.904,
 'epoch': 10.0}